In [1]:
# imports
import os, rasterio
import time
from deepforest import main, utilities, get_data, preprocess 
from os.path import exists
from  PIL import Image, ImagePath
Image.MAX_IMAGE_PIXELS = None
import matplotlib.pyplot as plt
import pandas as pd

c:\Users\Robeck\Downloads\df\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
# get and check Image
image_path=r"D:\Dropbox\P.Robeck\BPLA Dropbox\07 Temp\Hankos_Submittals\221001_DQ_ECW2022\img\DQ_2022_3b.tif"
# "D:\Dropbox\P.Robeck\BPLA Dropbox\07 Temp\Hankos_Submittals\221001_DQ_ECW2022\PNG\DQ_2022_3b.png"
print(exists(image_path))

# annotations = ("D:\\Dropbox\\P.Robeck\\BPLA Dropbox\\07 Temp\\Hankos_Submittals\\221001_DQ_ECW2022\\annotation\\deep-forest-training.csv")
# print(exists(annotations))

exImg=Image.open(image_path)
print(exImg.getbands())
print(exImg.size)
src = rasterio.open(image_path)

transform = src.transform 
crs = src.crs
print(crs)

True
('R', 'G', 'B')
(15652, 11592)
EPSG:32638


In [19]:
# get training bonding box and convert to the right format
ano_shp=r"D:\Dropbox\P.Robeck\BPLA Dropbox\03 Planning\DQ-Vegetation-Analysis-1180\03_Data\model-training\20230121-Five-classes-data-v1\5classes-treesdata-start-December2022-1.shp"
annotations=utilities.shapefile_to_annotations(ano_shp, image_path,  savedir='.')

In [20]:
# remove first column
annotations

,image_path,xmin,ymin,xmax,ymax,label
0,DQ_2022_3b.tif,9658,8365,9676,8380,Tree
1,DQ_2022_3b.tif,9672,8358,9689,8374,Tree
2,DQ_2022_3b.tif,9634,8335,9648,8348,Tree
3,DQ_2022_3b.tif,9631,8288,9641,8297,Tree
4,DQ_2022_3b.tif,9621,8287,9628,8294,Tree
...,...,...,...,...,...,...
587,DQ_2022_3b.tif,7827,2160,7850,2185,Tree
588,DQ_2022_3b.tif,9826,8165,9850,8190,Tree
589,DQ_2022_3b.tif,10733,7918,10755,7940,Tree
590,DQ_2022_3b.tif,10484,7038,10515,7065,Tree


In [21]:

ano_csv=r"D:\Dropbox\P.Robeck\BPLA Dropbox\03 Planning\DQ-Vegetation-Analysis-1180\03_Data\model-training\20230121-Five-classes-data-v1\1class-treesdata-start-December2022.csv"
annotations.to_csv(ano_csv)

In [22]:
print(pd.DataFrame(pd.read_csv(ano_csv,index_col=0)))

         image_path   xmin  ymin   xmax  ymax label
0    DQ_2022_3b.tif   9658  8365   9676  8380  Tree
1    DQ_2022_3b.tif   9672  8358   9689  8374  Tree
2    DQ_2022_3b.tif   9634  8335   9648  8348  Tree
3    DQ_2022_3b.tif   9631  8288   9641  8297  Tree
4    DQ_2022_3b.tif   9621  8287   9628  8294  Tree
..              ...    ...   ...    ...   ...   ...
587  DQ_2022_3b.tif   7827  2160   7850  2185  Tree
588  DQ_2022_3b.tif   9826  8165   9850  8190  Tree
589  DQ_2022_3b.tif  10733  7918  10755  7940  Tree
590  DQ_2022_3b.tif  10484  7038  10515  7065  Tree
591  DQ_2022_3b.tif   9956  4624   9978  4643  Tree

[592 rows x 6 columns]


In [23]:
#Write csv to file and crop to patches
tmpdir = (r"D:\Dropbox\P.Robeck\BPLA Dropbox\07 Temp\Hankos_Submittals\221001_DQ_ECW2022\crop")
# annotations.to_csv("{}/example.csv".format(tmpdir), index=False)
annotations_file = preprocess.split_raster(path_to_raster=image_path,
                                           annotations_file=ano_csv,
                                           base_dir=tmpdir,
                                           patch_size=500,
                                           patch_overlap=0.25)

# Returns a 6 column pandas array

c:\Users\Robeck\Downloads\df\.venv\lib\site-packages\deepforest\preprocess.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_annotations.image_path = "{}_{}.png".format(image_basename, index)
c:\Users\Robeck\Downloads\df\.venv\lib\site-packages\deepforest\preprocess.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_annotations.xmax = (selected_annotations.xmin - window_xmin) + (
c:\Users\Robeck\Downloads\df\.venv\lib\site-packages\deepforest\preprocess.py:112: SettingWithCopyWarning

In [24]:
annotations_file
del(annotations_file['Unnamed: 0'])
assert annotations_file.shape[1] == 6

In [25]:
annotations_file.to_csv(r"D:\Dropbox\P.Robeck\BPLA Dropbox\07 Temp\Hankos_Submittals\221001_DQ_ECW2022\crop\DQ_2022_3b.csv",index=0)


In [26]:
ano_split=(r"D:\Dropbox\P.Robeck\BPLA Dropbox\07 Temp\Hankos_Submittals\221001_DQ_ECW2022\crop\DQ_2022_3b.csv")
print(pd.DataFrame(pd.read_csv("D:\\Dropbox\\P.Robeck\\BPLA Dropbox\\07 Temp\\Hankos_Submittals\\221001_DQ_ECW2022\\crop\\DQ_2022_3b.csv")))
exists(ano_split)

             image_path  xmin  ymin  xmax  ymax label
0    DQ_2022_3b_323.png   479   385   500   411  Tree
1    DQ_2022_3b_324.png   479    10   500    36  Tree
2    DQ_2022_3b_354.png   104   385   128   411  Tree
3    DQ_2022_3b_354.png   280   345   312   377  Tree
4    DQ_2022_3b_355.png   104    10   128    36  Tree
..                  ...   ...   ...   ...   ...   ...
533  DQ_2022_3b_977.png    10   402    29   417  Tree
534  DQ_2022_3b_977.png     5   414    27   433  Tree
535  DQ_2022_3b_978.png    19    85    47   113  Tree
536  DQ_2022_3b_978.png    10    27    29    42  Tree
537  DQ_2022_3b_978.png     5    39    27    58  Tree

[538 rows x 6 columns]


True

In [27]:

ano_split
# os.path.dirname(ano_split)

'D:\\Dropbox\\P.Robeck\\BPLA Dropbox\\07 Temp\\Hankos_Submittals\\221001_DQ_ECW2022\\crop\\DQ_2022_3b.csv'

In [28]:
model = main.deepforest()
model.use_release()

# Example run with short training
model.config["train"]['epochs'] = 5
model.config['gpus'] = '-1' #move to GPU and use all the GPU resources
# model.config['cpu'] = 'accelerator'
model.config["save-snapshot"] = False
model.config["train"]["csv_file"] = ano_split
model.config["train"]["root_dir"] = os.path.dirname(ano_split)
model.config['num_worker']=16
# model.config['max_epochs']=10
model.create_trainer()



Reading config file: c:\Users\Robeck\Downloads\df\.venv\lib\site-packages\deepforest\data\deepforest_config.yml


c:\Users\Robeck\Downloads\df\.venv\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:467: LightningDeprecationWarning: Setting `Trainer(gpus='-1')` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices='-1')` instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Model from DeepForest release https://github.com/weecology/DeepForest/releases/tag/1.0.0 was already downloaded. Loading model from file.
Loading pre-built model: https://github.com/weecology/DeepForest/releases/tag/1.0.0


In [29]:
start_time = time.time()
model.trainer.fit(model)
print(f"--- Training on CPU: {(time.time() - start_time):.2f} seconds ---")

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type      | Params
------------------------------------
0 | model | RetinaNet | 32.1 M
------------------------------------
31.9 M    Trainable params
222 K     Non-trainable params
32.1 M    Total params
128.592   Total estimated model params size (MB)


c:\Users\Robeck\Downloads\df\.venv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:366: UserWarning: One of given dataloaders is None and it will be skipped.
  rank_zero_warn("One of given dataloaders is None and it will be skipped.")
c:\Users\Robeck\Downloads\df\.venv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 4: 100%|██████████| 137/137 [00:44<00:00,  3.10it/s, loss=0.768]

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 137/137 [00:44<00:00,  3.10it/s, loss=0.768]
--- Training on CPU: 220.55 seconds ---


In [ ]:
image_path

In [30]:
# predict to the image
# boxes = model.predict_image(path=image_path, return_plot = True)
predicted_raster = model.predict_tile(image_path, return_plot = False, patch_size=500, patch_overlap=0.25)

100%|██████████| 1302/1302 [02:44<00:00,  7.91it/s]


26946 predictions in overlapping windows, applying non-max supression
16689 predictions kept after non-max suppression


In [31]:
pd.DataFrame(predicted_raster)

,xmin,ymin,xmax,ymax,label,score,image_path
0,8522.0,7591.0,8542.0,7605.0,Tree,0.741371,DQ_2022_3b.tif
1,8242.0,1215.0,8262.0,1230.0,Tree,0.717726,DQ_2022_3b.tif
2,10924.0,6513.0,10944.0,6531.0,Tree,0.696795,DQ_2022_3b.tif
3,11646.0,6461.0,11671.0,6484.0,Tree,0.679135,DQ_2022_3b.tif
4,6394.0,3119.0,6414.0,3138.0,Tree,0.671558,DQ_2022_3b.tif
...,...,...,...,...,...,...,...
16684,3412.0,7678.0,3437.0,7698.0,Tree,0.100034,DQ_2022_3b.tif
16685,10242.0,4601.0,10249.0,4618.0,Tree,0.100032,DQ_2022_3b.tif
16686,11410.0,2906.0,11427.0,2922.0,Tree,0.100017,DQ_2022_3b.tif
16687,491.0,5350.0,499.0,5367.0,Tree,0.100004,DQ_2022_3b.tif


In [32]:
# transform to projected cooridinate system
r = rasterio.open(image_path)
transform = r.transform 
crs = r.crs
print(crs)

gdf = utilities.annotations_to_shapefile(predicted_raster, transform=transform, crs=crs)

EPSG:32638


c:\Users\Robeck\Downloads\df\.venv\lib\site-packages\deepforest\utilities.py:431: UserWarning: This method is deprecated and will be removed in version DeepForest 2.0.0, please use boxes_to_shapefile which unifies project_boxes and annotations_to_shapefile functionalities
  warnings.warn("This method is deprecated and will be removed in version DeepForest 2.0.0, please use boxes_to_shapefile which unifies project_boxes and annotations_to_shapefile functionalities")


In [33]:
gdf.to_file('DQ-trained-500-5e-5c.shp')

In [34]:
results = model.evaluate(ano_split, os.path.dirname(ano_split), iou_threshold = 0.45)

c:\Users\Robeck\Downloads\df\.venv\lib\site-packages\deepforest\IoU.py:88: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  matrix = overlap_df.pivot("truth_id", "prediction_id", "area").values
c:\Users\Robeck\Downloads\df\.venv\lib\site-packages\deepforest\IoU.py:88: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  matrix = overlap_df.pivot("truth_id", "prediction_id", "area").values
c:\Users\Robeck\Downloads\df\.venv\lib\site-packages\deepforest\IoU.py:88: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  matrix = overlap_df.pivot("truth_id", "prediction_id", "area").values
c:\Users\Robeck\Downloads\df\.venv\lib\site-packages\deepforest\IoU.py:88: FutureWarning: In a future version of pandas all arguments of DataFrame.pivot will be keyword-only.
  matrix = overlap_df.pivot("truth_id", "prediction_id", "area").values
c:\Users\Rob

In [35]:
print(results["box_recall"])
print(results["box_precision"])

0.7972112107706316
0.08346841294826486


In [36]:
results["results"].head()

,prediction_id,truth_id,IoU,score,xmin,xmax,ymin,ymax,predicted_label,true_label,image_path,match
0,3,0,0.662359,0.239328,479,500,385,411,Tree,Tree,DQ_2022_3b_323.png,True
0,2,0,0.685630,0.251145,479,500,10,36,Tree,Tree,DQ_2022_3b_324.png,True
0,7,0,0.599357,0.176266,104,128,385,411,Tree,Tree,DQ_2022_3b_354.png,True
1,3,1,0.826741,0.224218,280,312,345,377,Tree,Tree,DQ_2022_3b_354.png,True
0,11,0,0.568104,0.165652,104,128,10,36,Tree,Tree,DQ_2022_3b_355.png,True
